In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

from torch_geometric.datasets import ModelNet
import torch_geometric.transforms as T

import mlflow
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch_geometric.nn import DynamicEdgeConv, global_max_pool

from pointnet import PointNetCls
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score

In [2]:
import pptk

def view(points):
    v = pptk.viewer(points)
    v.attributes(points)
    v.set(point_size=0.01)
    # selected = points[v.get('selected')]
    return v

def plot_conf_matrix(conf_mtrx, labels, file_path='temp_conf_mtrx.png'):
    df_cm = pd.DataFrame(np.array(conf_mtrx), index=labels, columns=labels)
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig(file_path)
    plt.close()
    
def get_path_of_last_model():
    models_path = 'models'
    files = list(filter(lambda f: os.path.isfile(os.path.join(models_path, f)) and f.endswith('.pth'), os.listdir(models_path) ))
    if len(files) == 0:
        return None, 0
    files.sort(key=lambda f: int(f.split('.')[0].split('_')[-1] ))
    return os.path.join(models_path, files[-1]), int(files[-1].split('.')[0].split('_')[-1])

def test_model_full(classifier, test_data, num2cat, step=0, model_epoch_cumulatiove_base=0):
    all_labels = []
    all_choice = []
    for j, data in enumerate(test_loader, 0):
        points, labels = data
        points = points.transpose(2, 1)
        points, labels = points.to(device), labels.to(device)
        classifier = classifier.eval()
        with torch.no_grad():
            pred, _ = classifier(points)
        pred = pred.view(-1, num_classes)
        loss = F.nll_loss(pred, labels)
        pred_choice = pred.data.max(1)[1]
            
        all_labels.append(labels.cpu().numpy())
        all_choice.append(pred_choice.cpu().numpy())
            
    all_labels = np.concatenate(all_labels)
    all_choice = np.concatenate(all_choice)
    test_acc = accuracy_score(all_labels, all_choice)
    print(blue('epoch %d: %d/%d | test loss: %f | test acc: %f') % (model_epoch_cumulatiove_base+epoch+1, i+1, num_batch+1, loss.item(), test_acc))

    cnf_mtrx = confusion_matrix(all_labels, all_choice, labels=sorted(list(num2cat)))
    conf_mtrx_file_path = os.path.join("temp", f"test_cnf_mtrx_{epoch}_{i}.png")
    plot_conf_matrix(cnf_mtrx, [num2cat[num] for num in sorted(list(num2cat))], conf_mtrx_file_path)
    mlflow.log_artifact(conf_mtrx_file_path)
    mlflow.log_metric('test_acc', test_acc, step=step)
    return test_acc
    
def test_model_simple(model, test_loader, step=0):
    j, data = next(enumerate(test_loader, 0))
    points, labels = data
    points = points.transpose(2, 1)
    points, labels = points.to(device), labels.to(device)
    classifier = classifier.eval()
    with torch.no_grad():
        pred, _ = classifier(points)
    pred = pred.view(-1, num_classes)
    loss = F.nll_loss(pred, labels)
    pred_choice = pred.data.max(1)[1]
    correct = pred_choice.eq(labels.data).cpu().sum()
    test_acc = correct.item() / float(batchsize)
    print(blue('epoch %d: %d/%d | test loss: %f | test acc: %f') % (model_epoch_cumulatiove_base+epoch+1, i+1, num_batch+1, loss.item(), test_acc))

    # log test
    cnf_mtrx = confusion_matrix(labels.cpu().tolist(), pred_choice.cpu().tolist(), labels=sorted(list(num2cat)))
    conf_mtrx_file_path = os.path.join("temp", f"test_cnf_mtrx_{epoch}_{i}.png")
    plot_conf_matrix(cnf_mtrx, [num2cat[num] for num in sorted(list(num2cat))], conf_mtrx_file_path)
    mlflow.log_artifact(conf_mtrx_file_path)
    mlflow.log_metric('test_acc', np.mean(test_acc_epoch), step=step)
    return test_acc

In [3]:
batchsize = 32
blue = lambda x: '\033[94m' + x + '\033[0m'
yellow = lambda x: '\033[93m' + x + '\033[0m'
red = lambda x: '\033[91m' + x + '\033[0m'
pre_transform, transform = T.NormalizeScale(), T.SamplePoints(1024)
train_dataset = ModelNet('../data/modelnet10', '10', True, transform, pre_transform)
test_dataset = ModelNet('../data/modelnet10', '10', False, transform, pre_transform)

class Adapter:
    def __init__(self, pg_dataset):
        self.pg_dataset = pg_dataset

    def __len__(self):
        return len(self.pg_dataset)
    
    def __getitem__(self, idx):
        return self.pg_dataset[idx].pos.numpy(), self.pg_dataset[idx].y.numpy().item() 
    
print("train size: ", len(train_dataset), len(train_dataset)//batchsize)
print("test size: ", len(test_dataset), len(test_dataset)//batchsize)
train_loader = torch.utils.data.DataLoader(Adapter(train_dataset), batch_size=batchsize, shuffle=True, num_workers=0)
test_loader  = torch.utils.data.DataLoader(Adapter(test_dataset), batch_size=batchsize, shuffle=False, num_workers=0)
num2cat = dict(zip(range(10), train_dataset.raw_file_names))

train size:  3991 124
test size:  908 28


In [45]:
for i in range(3000):
    points, l = train_dataset[i].pos, train_dataset[i].y.item()
    if l == 0:
        break
        
print(num2cat[l], l, i)
view(T.RandomRotate(0, axis=2)(train_dataset[i]).pos.numpy())

bathtub 0 0


In [47]:
for i in range(3000):
    points, l = train_dataset[i].pos, train_dataset[i].y.item()
    if l == 0:
        break
        
print(num2cat[l], l, i)
view(T.RandomShear(0.5)(train_dataset[i]).pos.numpy())

bathtub 0 0


In [50]:
for i in range(3000):
    points, l = train_dataset[i].pos, train_dataset[i].y.item()
    if l == 0:
        break
        
print(num2cat[l], l, i)
view(T.RandomFlip(axis=1, p=1.0)(train_dataset[i]).pos.numpy())

bathtub 0 0
